In [ ]:
import numpy as np
import pandas as pd
import ipympl
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib
from echopop import Survey
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
from echopop.spatial.variogram import get_variogram_arguments
import numpy as np
import copy
from echopop.spatial.transect import edit_transect_columns
from echopop.spatial.projection import transform_geometry
from echopop.spatial.variogram import get_variogram_arguments
from echopop.analysis import variogram_analysis
from echopop.spatial.mesh import griddify_lag_distances
from echopop.spatial.variogram import empirical_variogram
from echopop import Survey
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
from echopop.spatial.variogram import get_variogram_arguments

In [ ]:

survey = Survey(init_config_path="C:/Users/Brandyn/Documents/GitHub/echopop/config_files/initialization_config.yml" ,
                survey_year_config_path="C:/Users/Brandyn/Documents/GitHub/echopop/config_files/survey_year_2019_config.yml")
survey.transect_analysis()
self = survey
settings_dict = {"variable": "biomass_density",
                 "stratum_name": "stratum_num",
                 "kriging_parameters": {
                    "longitude_reference": -124.78338,
                    "latitude_offset": 45.0,
                    "longitude_offset": -124.78338,
                 }}
# Prepare the transect data
# ---- Create a copy of the transect dictionary
transect_input = copy.deepcopy(self.analysis["transect"])
# ---- Edit the transect data
transect_data = edit_transect_columns(transect_input, settings_dict)
isobath_df = self.input["statistics"]["kriging"]["isobath_200m_df"]
transect_data, _, _ = transform_geometry(transect_data, isobath_df, settings_dict)

# Create a copy of the existing variogram settings
variogram_parameters = self.input["statistics"]["variogram"]["model_config"].copy()
n_lags = 30
lag_resolution = variogram_parameters["lag_resolution"]
# ---- Number of lags
variogram_parameters["n_lags"] = 30
# ---- Azimuth range
variogram_parameters["azimuth_range"] = 360
# ---- Force lag-0
variogram_parameters["force_lag_zero"] = True
# Compute the lag distances
distance_lags = np.arange(1, n_lags) * lag_resolution
# ---- Add to the `variogram_parameters` dictionary
variogram_parameters["distance_lags"] = distance_lags
# ---- Update the max range parameter, if necessary
max_range = lag_resolution * n_lags
lags, gamma_h, lag_counts, lag_covariance = empirical_variogram(
    transect_data, variogram_parameters, {"variable": "biomass_density"}
)

In [125]:
# Define the widget map
VARIOGRAM_MODEL_WIDGET_MAP = {
    "Bessel-exponential": ["bessel", "exponential"],
    "Bessel": "bessel",
    "Exponential": "exponential",
    "Gaussian": "gaussian",
    "Linear": "linear",
    "Sinc": "sinc",
    "Spherical": "spherical",
    "Bessel-Gaussian": ["bessel", "gaussian"],
    "Cosine-exponential": ["cosine", "exponential"],
    "Cosine-Gaussian": ["cosine", "gaussian"],
    "Exponential-linear": ["exponential", "linear"],
    "Gaussian-linear": ["gaussian", "linear"]
}

param_key = {
    "nugget": dict(name="Nugget", widget="entry", step=0.01),
    "sill": dict(name="Sill", widget="entry", step=0.01),
    "correlation_range": dict(name="Correlation range", widget="entry", step=0.0001),
    "decay_power": dict(name="Decay power", widget="entry", step=0.05),
    "hole_effect_range": dict(name="Hole effect range", widget="entry", step=0.0001),
    "enhance_semivariance": dict(name="Enhance semivariance", widget="checkbox", step=np.nan)
}

def get_variogram_defaults(survey, argument):
    DEFAULT_PARAMETERS = {
        "nugget": 0.0,
        "sill": 1.0,
        "hole_effect_range": 0.0,
        "decay_power": 1.5,
        "enhance_semivariance": False,
    }
    # ---- get variogram config
    if "model_config" in survey.input["statistics"]["variogram"]:
        if argument in survey.input["statistics"]["variogram"]["model_config"]:
            return survey.input["statistics"]["variogram"]["model_config"][argument]
        else:
            return DEFAULT_PARAMETERS[argument]

def get_settings_defaults(survey): 
    DEFAULT_PARAMETERS = {
        "n_lags": 30,
        "lag_resolution": 0.002,
        "azimuth_range": 360.0
    }

# Function to update variogram parameters based on dropdown selection
def update_variogram_params(change):
    with output_variogram_params:
        clear_output(wait=True)
        selected_model = change['new']
        print(f"Selected Model Type: {type(selected_model)}")
        print(f"Selected Model Value: {selected_model}")

        model_names = VARIOGRAM_MODEL_WIDGET_MAP[selected_model]
        print(f"{model_names}")

        function_arguments, _ = get_variogram_arguments(model_names)
        # ---- convert to a dictionary
        args = dict(function_arguments)

        print(f"{args.keys()}")

        # Clear existing widgets (if any)
        output_variogram_params.clear_output()
        variogram_widgets.clear()
        
        variogram_widgets["model"] = model_names

        # Display widgets for each parameter
        for param in args.values():
            arg = param.name

            if arg != "distance_lags":  # Adjust as needed
                default_value = get_variogram_defaults(survey, arg)
                # ---- get the name from the key
                arg_name = param_key[arg]["name"]
                widget_type = param_key[arg]["widget"]
                step_size = param_key[arg]["step"]

                if widget_type == "checkbox":
                    checkbox = widgets.Checkbox(value=default_value, description=arg_name, style={'description_width': 'initial'}, step=step_size)
                    variogram_widgets[arg] = checkbox
                    display(checkbox)
                # Example: Create widgets (Checkbox and FloatText) for each parameter
                elif widget_type == "entry":
                    entry = widgets.FloatText(value=default_value, description=arg_name, style=style, layout=layout)
                    variogram_widgets[arg] = entry
                    display(entry)

def plot_variogram():
    s = lambda x : (((x-x.min())/float(x.max()-x.min())+1)*10)**2
    x = lags
    y = gamma_h

    fig, ax = plt.subplots(figsize=(8, 5))
    scatter = ax.scatter(x, y, s=s(lag_counts), edgecolors='black')
    ax.set_xlabel('Lag Distance (h)')
    ax.set_ylabel('Semivariance (γ)')
    ax.set_title('Empirical Variogram')
    ax.set_axisbelow(True)
    ax.grid(True)
    plt.tight_layout()

    # Create a list of annotated points
    annot = ax.annotate("", xy=(0, 0), xytext=(-50, -50), textcoords="offset points",
                        bbox=dict(boxstyle="round", fc="white", ec="black", alpha=1.0),
                        arrowprops=dict(arrowstyle="->", color="black", lw=1, linestyle="-"))
    annot.set_visible(False)

    def update_annot(ind):
        pos = scatter.get_offsets()[ind["ind"][0]]
        annot.xy = pos
        text = f"h={pos[0]:.2f}\nγ={pos[1]:.2f}\nCount={lag_counts[ind['ind'][0]]}"
        annot.set_text(text)
        annot.get_bbox_patch().set_alpha(1.0)

    def on_hover(event):
        vis = annot.get_visible()
        if event.inaxes == ax:
            cont, ind = scatter.contains(event)
            if cont:
                update_annot(ind)
                annot.set_visible(True)
                fig.canvas.draw_idle()
            else:
                if vis:
                    annot.set_visible(False)
                    fig.canvas.draw_idle()

    fig.canvas.mpl_connect("motion_notify_event", on_hover)

    return fig, ax

def compute_theoretical_variogram(parameters):
    # Get the current input values
    arg_dict = {
        key: value.value if isinstance(value, (widgets.FloatText, widgets.Checkbox)) else value
        for key, value in parameters.items()
    }

    # pass into variogram
    from echopop.spatial.variogram import variogram
    distance_lags = lags
    return variogram(distance_lags, arg_dict)

def plot_theoretical_variogram(fig, ax, parameters):
    theoretical_variogram = compute_theoretical_variogram(parameters)
    
    ax.plot(lags, theoretical_variogram, linewidth = 3.0, c = "black")
    # plt.plot(lags, theoretical_variogram)
    fig.canvas.draw_idle()

def on_compute_button_clicked(button):
    parameters = variogram_widgets
    with out: 
        out.clear_output(wait=True)  # Clear previous plot
        fig, ax = plot_variogram()
        plot_theoretical_variogram(fig, ax, parameters)
        fig.canvas.toolbar_visible = False
        fig.canvas.header_visible = False
        fig.canvas.footer_visible = False
        fig.canvas.resizable = False
        plt.show(fig)
        # plt.close(fig)


In [126]:

import warnings
from IPython.display import display, HTML
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib widget
# DEFINE LAYOUT
style = {'description_width': '150px'}
layout = {'width': '400px'}

tab = widgets.Tab()

# Create widgets for Variogram Parameters tab
dropdown_variogram_model = widgets.Dropdown(
    options=list(VARIOGRAM_MODEL_WIDGET_MAP.keys()),
    value="Bessel-exponential",
    description='Variogram Model',
    disabled=False,
    style = {'description_width': '153px'}, 
    layout = {'width': '403px'}
)

# Initialize the variogram widgets dictionary
variogram_widgets = {}
output_variogram_params = widgets.Output()

# Attach the observer function to the dropdown
dropdown_variogram_model.observe(update_variogram_params, names='value')
update_variogram_params({'new': dropdown_variogram_model.value})
placeholder_optimization = widgets.Label(value='Optimization Parameters tab will go here.')

# Create a button to compute the theoretical variogram
button_compute_variogram = widgets.Button(description='Compute Theoretical Variogram',
                                          layout={"width": "405px"},
                                          style={'font_weight': 'bold'})
button_compute_variogram.on_click(on_compute_button_clicked)

tab_contents_variogram = widgets.VBox([
    dropdown_variogram_model, 
    output_variogram_params, 
    button_compute_variogram
    ])


tab_contents_optimization = widgets.VBox([placeholder_optimization])
DEFAULT_PARAMETERS = {
    "n_lags": 30,
    "lag_resolution": 0.002,
    "azimuth_range": 360.0
}
n_lags_text = widgets.FloatText(value=DEFAULT_PARAMETERS["n_lags"], description='Number of Lags', step=1, style=style, layout=layout)
lag_resolution_text = widgets.FloatText(value=DEFAULT_PARAMETERS["lag_resolution"], description='Lag Resolution', step=DEFAULT_PARAMETERS["lag_resolution"]/2, style=style, layout=layout)
azimuth_range_text = widgets.FloatText(value=DEFAULT_PARAMETERS["azimuth_range"], description='Azimuth Range', step=DEFAULT_PARAMETERS["azimuth_range"]/360, style=style, layout=layout)
tab_contents_settings = widgets.VBox([
    n_lags_text,
    lag_resolution_text,
    azimuth_range_text
])
accordion_layout = {'width': '450px'}
tab = widgets.Accordion(layout=accordion_layout)
tab.children = [tab_contents_settings, tab_contents_variogram, tab_contents_optimization]

# Set titles for tabs using Markdown for better formatting
tab.set_title(0, 'Empirical variogram', )
tab.set_title(1, 'Theoretical variogram', )
tab.set_title(2, 'Optimize variogram parameters')
# Output widget for plot
out = widgets.Output(layout=widgets.Layout(height='550px'))
with out:
    fig, ax = plot_variogram()
    plt.show(fig)
    
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False
# Display the tabs
display(widgets.HBox([tab, out]))

In [127]:
variogram_widgets

{'model': ['bessel', 'exponential'],
 'nugget': FloatText(value=0.0, description='Nugget', layout=Layout(width='400px'), style=DescriptionStyle(description_width='150px')),
 'sill': FloatText(value=0.91, description='Sill', layout=Layout(width='400px'), style=DescriptionStyle(description_width='150px')),
 'correlation_range': FloatText(value=0.007, description='Correlation range', layout=Layout(width='400px'), style=DescriptionStyle(description_width='150px')),
 'decay_power': FloatText(value=1.5, description='Decay power', layout=Layout(width='400px'), style=DescriptionStyle(description_width='150px')),
 'hole_effect_range': FloatText(value=0.0, description='Hole effect range', layout=Layout(width='400px'), style=DescriptionStyle(description_width='150px'))}

In [ ]:
parameters = {'model': model_name}
parameters = {arg_name: widget.value for arg_name, widget in variogram_widgets.items()}
parameters


In [ ]:
def compute_theoretical_variogram(parameters):
    # Get the current input values
    arg_dict = {
        key: value.value if isinstance(value, (widgets.FloatText, widgets.Checkbox)) else value
        for key, value in parameters.items()
    }

    # pass into variogram
    from echopop.spatial.variogram import variogram
    distance_lags = lags
    return variogram(distance_lags, arg_dict)

def plot_theoretical_variogram(parameters):
    theoretical_variogram = compute_theoretical_variogram(parameters)
    
    plt.plot(lags, theoretical_variogram)
    fig.canvas.draw_idle()

def on_compute_button_clicked(button):
    parameters = variogram_widgets
    plot_theoretical_variogram(parameters)